In [1]:
import numpy as np
from scipy.stats import levy_stable
from scipy.stats import uniform
from scipy.stats import multivariate_normal

In [2]:
def prior_sample():
    # Sample from prior distribution
    # For univariate alpha-stable models, sample each parameter from its respective uniform distribution
    #np.random.seed(seed)
    alpha = uniform.rvs(1.1, 0.9)  # U[1.1, 2]
    beta = uniform.rvs(-1, 2)  # U[-1, 1]
    gamma = uniform.rvs(0, 300)  # U[0, 300]
    delta = uniform.rvs(-300, 600)  # U[-300, 300]
    return [alpha, beta, gamma, delta]

In [3]:
def univ_alpha_stable_sampler(params,size,seed) :
    alpha,beta,gamma,delta = params[0],params[1],params[2],params[3]
    #print(alpha,beta,gamma,delta)
    y_bar = 0
    np.random.seed(seed)
    w = np.random.standard_exponential(size=size)
    u = np.random.uniform(low = -np.pi/2, high = np.pi/2, size = size)
    if alpha == 1 :
        y_bar = 2/np.pi*((np.pi/2+beta*u)*np.tan(u)-beta*np.log((np.pi/2*w*np.cos(u))/(np.pi/2+beta*u)))
        return gamma*y_bar + delta
    else :
        S = (1+beta**2*np.tan(np.pi*alpha/2)**2)**(1/(2*alpha))
        B = 1/alpha*np.arctan(beta*np.tan(np.pi*alpha/2))
        #print(S,B)
        y_bar = (S*(np.sin(alpha)*(u+B))*(np.cos(u-alpha*(u+B))/w)**((1-alpha)/alpha))/np.cos(u)**(1/alpha)
        return gamma*y_bar + delta

In [4]:
def compute_quantiles(data):
    # Compute quantile-based summary statistics
    quantiles = np.percentile(data, [5, 25, 50, 75, 95])
    v_alpha = (quantiles[4] - quantiles[0]) / (quantiles[3] - quantiles[1])
    v_beta = (quantiles[4] + quantiles[0] - 2 * quantiles[2]) / (quantiles[4] - quantiles[0])
    v_gamma = (quantiles[3] - quantiles[1]) / quantiles[2]
    return v_alpha, v_beta, v_gamma

In [5]:
def S(data):
    # Compute summary statistics of the data
    # This function should return low-dimensional summary statistics S(data)
    # For univariate alpha-stable models, use quantile-based estimators along with the mean of the data
    v_alpha, v_beta, v_gamma = compute_quantiles(data)
    mean_x = np.mean(data)
    return np.array([v_alpha, v_beta, v_gamma, mean_x])

In [6]:
def psi(y, x, covariance = np.diag([0.25,0.25,1,1])):
    # Define smoothing kernel using a Gaussian kernel with estimated covariance
    return multivariate_normal.pdf(y, mean=x, cov=covariance)


In [7]:
def cov_estimate(theta_hat,n_draws = 1000):
    x = np.array([univ_alpha_stable_sampler(params = theta_hat, size = 200, seed=None) for _ in range(n_draws)])
    sumary_statistics = np.array([S(x_i) for x_i in x])

    return np.cov(sumary_statistics, rowvar=False)

In [8]:
def norm2(u) :
    return np.sqrt(np.sum(u*u))


In [10]:
def K(u, sigma) :
    d = sigma.shape[0]
    #print(d)
    return np.exp(-(u.T@np.linalg.inv(sigma)@u)/2)/(np.sqrt(2*np.pi)**d*np.sqrt(np.linalg.det(sigma)))




In [ ]:
a = np.random.randint(0,10,4)
a

array([9, 7, 2, 5])

In [ ]:
K(a, np.diag([0.25,0.25,1,1]))

6.192485126867338e-121

In [ ]:
b = np.random.randint(0,10,4)
b

array([5, 7, 3, 5])

In [ ]:
K(b, np.diag([0.25,0.25,1,1]))

2.2238768838313724e-73

In [11]:
def resample(theta, weights,N) :
    normalized_weights = weights['t']/np.sum(weights['t'])
    print(theta['t-1'].shape)
    new_indices = np.random.choice(range(N),replace=True, p=normalized_weights)
    theta['t'] = theta['t'][new_indices]

    weights['t'] = np.ones(N)/N

In [ ]:
a = np.random.randint(0,10,10)
a

array([6, 9, 7, 7, 9, 6, 1, 5, 8, 5])

In [ ]:
a.shape

(10,)

In [12]:
def sample_from_M_t(N, theta_prev, weights_prev):
    normalized_weights = weights_prev/np.sum(weights_prev)
    # Choose a component based on the weights
    component = np.random.choice(range(N), p=normalized_weights)
    # Generate a sample from the chosen component
    sample = np.random.multivariate_normal(mean = theta_prev[component], cov = np.diag([0.25,0.25,1,1]))
    #samples[i] = sample
    return sample 

In [13]:
def M_t(N,theta_i,theta_prev, weights_prev) :
    densities = np.array([ psi(theta_i, theta_prev[i]) for i in range(N)])
    return np.sum(weights_prev*densities)
    

In [14]:
def normalize(u) :
    return u/np.sum(u)


In [15]:
def prior_density(theta_i) :
    alpha,beta,gamma,delta = theta_i[0],theta_i[1],theta_i[2],theta_i[3]
    return uniform.pdf(alpha,1.1,0.9)*uniform.pdf(beta,-1,2)*uniform.pdf(gamma,0,300)*uniform.pdf(delta,-300,600)

In [38]:
round(0.001,5)

0.001

In [39]:
def SMC_sampler_v2(arg, T=10,N =1000,n=200,covariance = np.diag([0.25, 0.25, 1, 1]), seed=42) :
    #epsilons = [k for k in range(1000,100, -100)]+[k for k in range(100,9, -1)] + [k+0.5 for k in range(9,4,-1)] + [5-k*0.05 for k in range(40)]+[3-0.01*k for k in range(301)]  # Tolerance schedule
    epsilons = [100*k for k in range(10,0,-1)]
    T = len(epsilons)

    #y = levy_stable.rvs(1.7, 0.9, loc=10, scale=10, size=200)
    ###--------Initialization :
    epsilon = epsilons[0]
    # 1 - sample theta from the prior : 
    # theta is a Nx4 matrix containing all the N theta_i
    #initial_theta = np.array([prior_sample(seed) for _ in range(N)])
    #theta = {"t" : initial_theta , "t-1" :initial_theta }
    # set weights their initial values :
    #x = np.array([univ_alpha_stable_sampler(params=theta['t'][i], size=n, seed=seed) for i in range(N)])
    #print('x.shape = ',x.shape)
    #initial_weights = np.array([K((S(y)-S(x))/epsilon,sigma_hat(theta_hat))/epsilon for theta_hat in theta['t']])
    #print("weights = ", initial_weights)
    #print(" sum(weights) = ", np.sum(initial_weights))
    #weights = {"t" : initial_weights, "t-1" : initial_weights}
    y,x,theta,weights,sigma_hat = arg
    t=1
    res = []
    sigma_hat = cov_estimate(np.array([1.7,0.9,10,10]))
    """
    weigths['t'] = np.ones(N)/N
    weights['t-1'] = np.ones(N)/N
    """
    print("INITIALIZATION IS DONE :)")
    while t<T :
        print("t = ",t)
        epsilon = epsilons[t]
        print("epsilon = ",epsilon)
        theta_maj = theta['t']
        weights_maj = weights['t']
        i = 0 
        #max_iter = 10
        #nb_iter = 0
        c_t = np.quantile(weights['t'], 0.9)
        while i<N : 
            #print('###########    i = ',i)
            """
            weights['t'] = normalize(weights['t'])
            weights['t-1'] = normalize(weights['t-1'])
            """
            #c_t = np.quantile(weights['t'], 0.9)

            #print("nb_iter = ",nb_iter)
            # Mutation and correction : 
            theta['t'][i] = sample_from_M_t(N, theta['t-1'], weights['t-1'])
            #print(f'theta["t"][{i}]= ', np.round(theta['t'][i],3))
            #sigma_hat = cov_estimate(np.array([1.7,0.9,10,10]))
            weights['t'][i] = (K((S(y)-S(x[i]))/epsilon,(epsilon**2)*sigma_hat)/epsilon)*prior_density(theta['t'][i])/M_t(N,theta['t'][i],theta['t-1'],weights['t-1'])
            print(f"prior(theta_{i}) = {prior_density(theta['t'][i])}")
            print(f"noyau = {K((S(y)-S(x[i]))/epsilon,(epsilon**2)*sigma_hat)/epsilon}")
            print(f" M_t = {M_t(N,theta['t'][i],theta['t-1'],weights['t-1'])}")
            """
            weights['t'] = normalize(weights['t'])
            weights['t-1'] = normalize(weights['t-1'])
            """
            #print('weights["t"] = ', np.round(weights['t'][:5],3))
            #theta['t-1'] = theta_maj
            #weights['t-1'] = weights_maj
            u = np.random.uniform(low=0, high=1, size=1)
            #c_t = np.quantile(weights['t'], 0.9)
            proba = 1-np.minimum(1,weights['t']/c_t)
            if t>1 : 
                print("c_t = ", c_t)
                print(f"weights['t'][{i}] = ", np.round(weights['t'][i],5))
                print('sum(weights) = ', np.sum(weights['t']))
                print(f'proba[{i}] = ', proba[i])
            """
            if nb_iter == max_iter: 
                i=i+1
                n_iter = 0
                continue
            """
            if u<=proba[i] :
                #Rejection : 
                print(f"theta['t'][{i}] = {np.round(theta['t'][i],3)} rejected !")
                #nb_iter+=1
                continue
            else : 
                # Acceptance
                print(f"theta['t'][{i}] = {np.round(theta['t'][i],3)} accepted !")
                weights['t'][i] = weights['t'][i]/(1-proba[i])
                i = i+1
                #nb_iter = 0
        t+=1
        theta['t-1'],weights['t-1'] = theta_maj,weights_maj
        res.append((theta['t'], weights['t']))
        # Resample : 
        weights['t'] = np.ones(N)/N
        weights['t-1'] = np.ones(N)/N
        print("theta = ", theta['t'][:5])
        print("weights = ", weights['t'][:5])


In [35]:
def initialization(T=10,N =1000,n=200,covariance = np.diag([0.25, 0.25, 1, 1]), seed=42) :
    #epsilons = [k for k in range(1000,100, -100)]+[k for k in range(100,9, -1)] + [k+0.5 for k in range(9,4,-1)] + [5-k*0.05 for k in range(40)]+[3-0.01*k for k in range(301)]  # Tolerance schedule
    epsilons = [100*k for k in range(10,0,-1)]
    T = len(epsilons)
    y = levy_stable.rvs(1.7, 0.9, loc=10, scale=10, size=200)
    ###--------Initialization :
    epsilon = epsilons[0]
    
    #sigma_hat = np.diag([0.25,0.25,1,1])
    sigma_hat = cov_estimate(np.array([1.7,0.9,10,10]))
    print("sigma_hat = ", sigma_hat)
    print('det(sigma_hat) = ', np.linalg.det(sigma_hat))
    # 1 - sample theta from the prior : 
    # theta is a Nx4 matrix containing all the N theta_i
    initial_theta = np.array([prior_sample() for _ in range(N)])
    theta = {"t" : initial_theta , "t-1" :initial_theta }
    print('theta["t"] = ',np.round(theta['t'][:5],4))
    # set weights their initial values :
    x = np.array([univ_alpha_stable_sampler(params=theta['t'][i], size=n, seed=None) for i in range(N)])
    print('x.shape = ',x.shape)
    print('x',x)
    #print([S(x[i]) for i in range(5)])
    #print([K((S(y)-S(x[i]))/epsilon,sigma_hat)/epsilon for i in range(5)])
    initial_weights = np.array([K((S(y)-S(x[i]))/epsilon,epsilon**2*sigma_hat)/epsilon for i in range(N)])
    print("weights = ", initial_weights)

    #print(" sum(weights) = ", np.sum(normalize(initial_weights)))

    #weights = {"t" : normalize(initial_weights), "t-1" : normalize(initial_weights)}
    weights = {"t" : initial_weights, "t-1" : initial_weights}
    return y,x,theta,weights,sigma_hat

In [40]:
arg = initialization()

sigma_hat =  [[ 2.67891191e+00 -1.11532813e-01 -1.57806390e-01 -3.81140911e+00]
 [-1.11532813e-01  2.11792024e-02 -2.46760592e-02  7.71412422e-01]
 [-1.57806390e-01 -2.46760592e-02  3.11217500e-01 -2.52485081e+00]
 [-3.81140911e+00  7.71412422e-01 -2.52485081e+00  1.48232119e+03]]
det(sigma_hat) =  15.53338335765651
theta["t"] =  [[ 1.135800e+00 -7.654000e-01  2.974370e+01  1.540489e+02]
 [ 1.345500e+00 -2.363000e-01  2.174903e+02  6.945790e+01]
 [ 1.522800e+00 -2.090000e-02  1.141824e+02  2.719215e+02]
 [ 1.280200e+00  1.517000e-01  1.339824e+02  9.152300e+00]
 [ 1.135600e+00  7.003000e-01  3.123720e+01  2.589335e+02]]
x.shape =  (1000, 200)
x [[2366.20985641  634.00538019  496.50856907 ...  263.9526689
   -10.55264884 2616.01155236]
 [  68.23912481   49.52645608  628.12932738 ... 1100.82278137
     3.77441605  170.05119073]
 [ 262.89807623  453.16967698  188.62468334 ...  267.6835287
   237.43063689  428.74613023]
 ...
 [-308.94436225  -73.34305284  143.58824215 ...  178.81927434
   

In [41]:
SMC_sampler_v2(arg=arg)

INITIALIZATION IS DONE :)
t =  1
epsilon =  900
prior(theta_0) = 3.0864197530864204e-06
noyau = 2.957721077526992e-17
 M_t = 1.2269558789377168e-05
theta['t'][0] = [ 1.67800e+00 -2.07000e-01  2.09213e+02 -5.67050e+01] accepted !
prior(theta_1) = 0.0
noyau = 2.9577210829901346e-17
 M_t = 1.6568954525913349e-06
theta['t'][1] = [  2.503   0.287 208.838 -56.306] rejected !
prior(theta_1) = 3.0864197530864204e-06
noyau = 2.9577210829901346e-17
 M_t = 1.4668257003904955e-06
theta['t'][1] = [  1.744  -0.268 211.266 -56.673] accepted !
prior(theta_2) = 0.0
noyau = 2.9577210824132215e-17
 M_t = 5.536607595266685e-06
theta['t'][2] = [  2.218  -0.393 208.686 -56.788] rejected !
prior(theta_2) = 3.0864197530864204e-06
noyau = 2.9577210824132215e-17
 M_t = 2.5236882666125146e-06
theta['t'][2] = [  1.851  -0.243 209.207 -54.962] accepted !
prior(theta_3) = 3.0864197530864204e-06
noyau = 2.957721061798833e-17
 M_t = 2.234521064885589e-06
theta['t'][3] = [ 1.55700e+00  6.70000e-02  2.08999e+02 -5.4973

KeyboardInterrupt: 